In [350]:
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [351]:
llm = ChatOllama(model="ggml-model-Q5_K_M/Modelfile:latest",callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [352]:
loader = PyPDFDirectoryLoader('./')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=50)
splits = text_splitter.split_documents(data)

Ignoring wrong pointing object 6 0 (offset 0)


In [353]:
embeddings = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask', model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True})
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 0.15})

c:\Users\emine\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [354]:
query = '자서전에 들어갈 프롤로그를 작성해줘'
docs = retriever.get_relevant_documents(query)

In [357]:
print(docs)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\

In [359]:
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

In [284]:
template = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
"""

In [285]:
prompt = ChatPromptTemplate.from_template(template)

1. 서론
2. 어린 시절과 가족 배경
3. 교육 여정 및 직장 생활
4. 개인적 관계와 사랑 이야기
5. 성취 및 인정
6. 도전과 고난 극복
7. 삶에서 가장 소중한 교훈
8. 결론: 미래에 대한 희망과 유산

In [286]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({'context': (format_docs(docs)), 'question':query})
print(response)

1. 서론
2. 어린 시절과 가족 배경
3. 교육 경험 및 전문적 성장
4. 주요 성과와 업적
5. 가장 큰 도전 과제들
6. 가치관과 신념
7. 개인적인 관계와 사랑 이야기
8. 멘토링과 지도에 대한 헌신
9. 사회에 미친 영향력
10. 결론 및 미래 세대에게 남기는 조언

이 목차들은 자서전의 주요 내용을 개요화하고, 독자들이 자신의 경험, 교훈, 그리고 유산에 대해 더 잘 이해할 수 있도록 도와줍니다.1. 서론
2. 어린 시절과 가족 배경
3. 교육 경험 및 전문적 성장
4. 주요 성과와 업적
5. 가장 큰 도전 과제들
6. 가치관과 신념
7. 개인적인 관계와 사랑 이야기
8. 멘토링과 지도에 대한 헌신
9. 사회에 미친 영향력
10. 결론 및 미래 세대에게 남기는 조언

이 목차들은 자서전의 주요 내용을 개요화하고, 독자들이 자신의 경험, 교훈, 그리고 유산에 대해 더 잘 이해할 수 있도록 도와줍니다.


In [287]:
query1 = 'PDF 문서룰 통해 자서전에 서론을 작성해줘'
docs1 = retriever.get_relevant_documents(query1)

In [288]:
template1 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [289]:
prompt1 = ChatPromptTemplate.from_template(template1)

In [290]:
chain1 = prompt1 | llm | StrOutputParser()
response1 = chain1.invoke({'context': (format_docs(docs1)), 'question':query1})
print(response1)

{\\textit{서론}}

{\\large 소중한 나와의 이야기, 그리고 인생의 여정}\n

당신의 삶은 수많은 사건들과 순간들로 가득 차 있습니다 - 기쁨, 슬픔, 고난과 승리의 순간들입니다. 당신의 이야기는 당신만을 위한 고유한 것으로, 당신이 경험하고 배운 모든 것들에 대한 증거입니다. 이러한 추억들을 되돌아보고 소중히 여기는 것은 우리 자신을 이해하고 앞으로 나아갈 방향을 결정하는 데 도움을 줍니다. 이 자서전을 통해 저는 제 이야기 - 나의 여정, 신념, 가치관, 그리고 저로 하여금 강인한 개인으로 성장시킨 교훈들을 여러분과 나누고자 합니다. 

저의 이야기는 단순한 나열이 아니라 제 삶을 형성해온 사건들과 경험들에 대한 감정적이고 개인적인 반영입니다. 우리는 모두 독특하며, 우리의 이야기를 공유하는 것은 연결고리를 만들고 다른 사람들과 공감할 수 있는 방법입니다. 저는 이 자서전이 저의 여정에 대해 더 깊이 이해하게 하고, 과거의 저를 이해하고 미래에 대비하는 데 도움이 될 것이라고 희망합니다.

저의 이야기는 어린 시절부터 시작되어, 저로 하여금 사랑과 인내, 열정의 가치를 소중히 여기도록 해준 가족으로부터 시작합니다. 대학 생활은 글쓰기와 문학에 대한 제 열정을 키운 시기였으며, 결혼 후 가족을 위해 더 이상 헌신적으로 살아가기 시작했습니다. 이 시기는 또한 인간관계와 협력의 중요성에 대해 귀중한 교훈을 가르쳐 주었습니다.

그 다음으로, 첫 직장에서의 경험과 인생에서 가장 큰 영향을 미친 사람들과의 만남에 대해 자세히 설명합니다. 이러한 경험들은 제 삶을 형성하는 데 도움을 준 순간들이며, 제가 오늘날 누구인지에 큰 역할을 했습니다. 또한 결혼 결정 같은 중요한 선택과 실패를 극복하고 성공을 이루기까지의 여정에 대해서도 이야기합니다.

제 이야기가 여러분에게 영감을 주고 새로운 관점을 열어주기를 바랍니다. 우리는 모두 독특하며, 우리만의 이야기를 가지고 있습니다. 이 자서전을 통해 저의 여정을 공유하고 제 삶을 형성한 교훈들을 나누며, 그것이

In [291]:
query2 = 'PDF 문서를 통해 어린 시절과 가족배경을 회고해줘'
docs2 = retriever.get_relevant_documents(query2)

In [292]:
print(docs2)

[Document(metadata={'page': 4, 'source': '자서전인터뷰.pdf'}, page_content='다.\x0053.\x00삶에서\x00가장\x00기쁜\x00순간은\x00언제였나요?\x00•첫\x00손자가\x00태어났을\x00때가\x00가장\x00기뻤습니다.\x00새로운\x00생명의\x00탄생을\x00보고,\x00제\x00가족이\x00다음\x00세대까지\x00이어지는\x00것을\x00느끼며\x00큰\x00행복을\x00느꼈습니다.\x0054.\x00가장\x00큰\x00좌절을\x00겪었을\x00때,\x00그\x00좌절을\x00이겨내기\x00위해\x00무엇을\x00했나요?\x00•좌절감을\x00이겨내기\x00위해\x00가족과\x00대화를\x00많이\x00나눴습니다.\x00또한,\x00그\x00시기를\x00어떻게\x00극복할\x00수\x00있을지\x00계획을\x00세우고,\x00작은\x00목표부터\x00차근차근\x00이뤄나가는\x00방법으로\x00극복했습니다.\x0055.\x00당신이\x00인생에서\x00배운\x00가장\x00중요한\x00교훈은\x00무엇인가요?\x00•인생에서\x00배운\x00가장\x00중요한\x00교훈은,\x00실패를\x00두려워하지\x00말고\x00계속\x00도전해야\x00한다는\x00것입니다.\x00실패는\x00성공으로\x00가는\x00길의\x00일부일\x00뿐이라는\x00것을\x00깨달았죠.\x0056.\x00삶에서\x00가장\x00행복했던\x00순간은\x00언제였나요?\x00•가족\x00모두가\x00함께\x00모여\x00따뜻한\x00저녁\x00식사를\x00하며\x00이야기를\x00나눴던\x00순간들이\x00가장\x00행복했습니다.\x00일상적인\x00순간이지만,\x00그\x00소중함이\x00얼마나\x00큰지\x00시간이\x00지날수록\x00더\x00느끼게\x00됩니다.\x0057.\x00삶에서'), Document(metadata={'page': 4, 'source': '자서전인터뷰.pdf'}, page_conten

In [293]:
template2 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [294]:
prompt2 = ChatPromptTemplate.from_template(template2)

In [295]:
chain2 = prompt2 | llm | StrOutputParser()
response2 = chain2.invoke({'context': (format_docs(docs2)), 'question':query2})
print(response2)

PDF 문서를 통해 어린 시절과 가족 배경을 회고해보면, 제 이름은 '영수'로 영원으로 삶을 이어받으라는 뜻으로 부모님이 지어주신 소중한 선물이었습니다. 어릴 적 기억으로는 눈이 내리는 것을 처음 보고 신나했던 일이나 어머니가 따뜻하게 감싸주시던 정이 느껴지는 경험들이 기억에 남아요. 아버지께서는 엄격하셨지만 가족을 위해 헌신하셨고, 어머니는 부드러우시며 정직과 성실을 강조하셨죠. 저는 시냇가에 있는 작은 숲을 좋아했는데, 그곳에서 친구들과 함께 자연 속에서 재미있게 지냈답니다. 형은 저를 잘 챙겨주었으며 때로는 엄하게 지도해주기도 했고요. 제가 가장 좋아하는 과목은 국어 시간이었는데, 책 읽고 글쓰기를 통해 세상을 경험하는 것처럼 느꼈습니다.

첫 번째 친구로는 철수라는 친구가 있었고 우리는 함께 연을 날리며 추운 날씨에도 열심히 높이 올리려 했던 추억이 있습니다. 어린 시절 저를 가장 많이 도와주신 분은 할머니로, 부모님이 일 때문에 바쁘실 때 저에게 이야기를 들려주시고 돌봐주셨어요. 어릴 적에는 나라를 지키고 싶어 군인이 되고자 하는 꿈이 있었습니다.

가족은 정직과 근면을 무엇보다 중요하게 여기며 성실히 일하고 남을 속이지 말라고 가르치셨습니다. 가정 형편이 넉넉하지 않아 공부나 활동에 제약이 있었지만, 이를 극복하며 더욱 강해져나가려는 의지를 키워주었죠. 어린 시절 꿈과 현실의 간극을 느끼면서도 그 경험들이 저를 더 강하게 만들었고 성장하게 했다고 생각합니다.

결론적으로, 제 이름 '영수'는 부모님의 사랑과 기대에서 비롯된 것으로, 어린 시절 소중한 추억들과 함께 저를 형성하는데 큰 역할을 했습니다. 가족 내에서의 가치관과 가치가 담긴 가르침들은 삶을 살아가는 데 있어 중요한 기초가 되었으며, 꿈과 현실의 간극을 극복하며 더욱 성장할 수 있는 원동력이 되어주었습니다.PDF 문서를 통해 어린 시절과 가족 배경을 회고해보면, 제 이름은 '영수'로 영원으로 삶을 이어받으라는 뜻으로 부모님이 지어주신 소중한 선물이었습니다. 어릴 적 기억으로는 눈이 내리는 

In [296]:
query3 = 'PDF 문서를 통해 교육경험 및 전문적 성장을 회고해줘'
docs3 = retriever.get_relevant_documents(query3)

In [297]:
print(docs3)

[Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과')]


In [298]:
template3 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [299]:
prompt3 = ChatPromptTemplate.from_template(template3)

In [300]:
chain3 = prompt3 | llm | StrOutputParser()
response3 = chain3.invoke({'context': (format_docs(docs3)), 'question':query3})
print(response3)

저는 언제나 다양한 경험을 추구하고 새로운 도전에 맞서는 것을 좋아했습니다. 이런 성향은 제 인생 여정 내내 저를 인도해주었고 결국에는 서울대학교에 입학하여 국제무역학을 전공하게 되었습니다. 학교 생활 동안 저는 많은 것을 배웠고 오늘날의 저를 형성하는 데 도움을 준 소중한 경험들을 했습니다.

서울대학교에서 보낸 시간들은 저에게 변혁적인 시기였습니다. 그곳에서 만난 사람들, 교수님들이 제 인생에 큰 영향을 미쳤으며, 그들의 가르침과 지도는 오늘날 제가 추구하는 열정적이고 책임감 있는 개인으로 만드는 데 기여했습니다. 국제무역학을 공부하며 저는 복잡한 경제 시스템을 이해하고 다양한 국가간의 비즈니스 관계를 탐구하는 법을 배웠습니다.

졸업 후, 저는 여러 회사에서 일하면서 제 전문성을 키우고 전문적인 성장을 이룰 수 있었습니다. 이 경험들은 제가 인내심, 책임감 및 회복력을 개발하는 데 도움이 되었으며 이러한 특성들은 저를 오늘날의 자리에 이르게 하는 데에 중요한 역할을 했습니다. 저는 또한 가족과의 소중한 시간을 즐길 기회도 가졌는데, 이는 삶의 진정한 의미를 되돌아보고 가족과 함께하는 것이 얼마나 귀중한지 깨닫게 해 주었습니다.

교육 여정을 통해 배운 것들은 제 인생 전반에 걸쳐 저에게 도움이 되었고, 오늘날 제가 추구하는 성공적이고 책임감 있는 개인으로 만드는 데 기여했습니다. 국제무역학 학위를 취득함으로써 저는 다양한 산업에서 일할 수 있는 필요한 지식과 기술을 얻었으며 이는 저를 직업 세계에서 두드러지게 만들 수 있게 했습니다.

저의 교육 경험과 전문적인 성장은 저에게 소중한 추억을 안겨주었을 뿐만 아니라 앞으로 나아가기 위한 도구와 자원을 제공해주었습니다. 저는 언제나 새로운 것들을 배우고 새로운 도전을 추구하는 것에 열려 있으며, 그것이 바로 제 인생 여정을 흥미롭고 보람차게 만드는 것입니다.저는 언제나 다양한 경험을 추구하고 새로운 도전에 맞서는 것을 좋아했습니다. 이런 성향은 제 인생 여정 내내 저를 인도해주었고 결국에는 서울대학교에 입학하

In [337]:
query4 = 'PDF 문서를 통해 인생의 성공과 업적을 회고해줘'
docs4 = retriever.get_relevant_documents(query4)

In [341]:
print(docs4)

[Document(metadata={'page': 6, 'source': '자서전 가상 100문100답.pdf'}, page_content='\x00덕분에\x00정말\x00행복한\x00인생을\x00살\x00수\x00있었어요.\x00앞으로도\x00우리\x00사랑이\x00계속\x00이어지기를\x00바라요.\x0083.\x00당신의\x00인생을\x00한\x00문장으로\x00표현한다면\x00무엇이라고\x00말하겠습니까?\x00•"사랑과\x00도전으로\x00가득한\x00인생"이라고\x00표현하고\x00싶습니다.\x00사랑하는\x00사람들과\x00함께한\x00시간,\x00그리고\x00끊임없는\x00도전이\x00저를\x00여기까지\x00이끌어\x00주었어요.\x0084.\x00지금까지의\x00삶을\x00돌아봤을\x00때,\x00가장\x00기억에\x00남는\x00일은\x00무엇인가요?\x00•첫\x00손자의\x00돌잔치가\x00가장\x00기억에\x00남아요.\x00그날의\x00기쁨과\x00손자의\x00미소가\x00너무나\x00소중한\x00추억으로\x00남아\x00있습니다.\x0085.\x00인생에서\x00가장\x00중요한\x00가치는\x00무엇이라고\x00생각하시나요?\x00•사랑과\x00신뢰입니다.\x00사람들과의\x00관계에서\x00이\x00두\x00가지\x00가치를\x00지키는\x00것이\x00인생을\x00행복하게\x00만들어\x00준다고\x00믿어요.\x0086.\x00당신이\x00가장\x00소중히\x00여기는\x00기억은\x00무엇인가요?\x00•아내와\x00함께한\x00첫\x00여행이\x00가장\x00소중한\x00기억이에요.\x00그때\x00느꼈던\x00자유와\x00행복감이\x00지금도\x00생생하게\x00떠오릅니다.\x0087.\x00인생에서\x00가장\x00용서하기\x00어려웠던\x00일은\x00무엇인가요?\x00'), Document(metadata={'page': 6, 'source': '자서전 가상 100문100답.pdf'}, page_content='\

In [342]:
template4 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [343]:
prompt4 = ChatPromptTemplate.from_template(template4)

In [344]:
chain4 = prompt4 | llm | StrOutputParser()
response4 = chain4.invoke({'context': (format_docs(docs4)), 'question':query4})
print(response4)

PDF 문서를 통해 인생의 성공과 업적에 대한 통찰을 공유할 수 있어 기쁩니다. 저의 인생 여정을 되돌아보면, 여러 도전들을 극복하고 성취한 많은 기억에 남는 순간들과 소중한 교훈들이 있었습니다.

첫째로, 사랑하는 가족과 친구들의 지지와 사랑이 제 성공의 원동력이었습니다. 그들은 어려운 시기에도 저를 격려해주고 인생의 중요한 단계마다 함께해주었습니다. 또한, 끊임없는 자기계발에 대한 헌신이 오늘날 제가 있는 위치에 오르는 데 결정적인 역할을 했습니다. 저는 새로운 기술들을 배우고, 도전 과제를 극복하며, 지속적으로 스스로를 밀어붙이는 데 전념해왔습니다.

제 경력에서 가장 기억에 남는 순간 중 하나는 첫 직장을 구했을 때였습니다. 그 직업은 저에게 꿈만 같았고, 제 능력에 확신을 갖고 뛰어난 성과를 거두었습니다. 또한, 제가 창업한 사업체로서 비즈니스 세계에서 성공을 거둔 것도 자랑스러운 일입니다.

인생에서 용서하기 어려웠던 일 중 하나는 친구와의 다툼이었습니다. 우리는 서로 다른 길을 걸었고, 결국 관계가 소원해졌습니다. 하지만 시간이 흐른 뒤에 저는 그 관계를 소중히 여겼음을 깨닫고 용서를 구했고, 우리의 유대가 얼마나 가치 있는지 깨달았습니다.

결론적으로, 인생에서 성공과 성취는 강한 관계, 지속적인 자기개발, 그리고 도전을 마주하고 극복하는 능력에 달려있다는 것을 배웠습니다. 이 교훈들은 제가 인생의 궤적을 형성하는데 도움을 주었고, 앞으로 계속 배울 것을 기대합니다.

이 통찰들이 여러분의 경험에 영감을 주고 소중한 인생 여정을 되돌아보는 데 도움이 되기를 바랍니다.PDF 문서를 통해 인생의 성공과 업적에 대한 통찰을 공유할 수 있어 기쁩니다. 저의 인생 여정을 되돌아보면, 여러 도전들을 극복하고 성취한 많은 기억에 남는 순간들과 소중한 교훈들이 있었습니다.

첫째로, 사랑하는 가족과 친구들의 지지와 사랑이 제 성공의 원동력이었습니다. 그들은 어려운 시기에도 저를 격려해주고 인생의 중요한 단계마다 함께해주었습니다. 또한, 끊임없는 자기계발에 대한

In [239]:
query5 = 'PDF 문서을 통해 업적, 성취 그리고 영향력 있는 순간들을 회고해줘'
docs5 = retriever.get_relevant_documents(query5)

In [240]:
print(docs5)

[Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010')]


In [241]:
template5 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [242]:
prompt5 = ChatPromptTemplate.from_template(template5)

In [243]:
chain5 = prompt5 | llm | StrOutputParser()
response5 = chain5.invoke({'context': (format_docs(docs5)), 'question':query5})
print(response5)

제 삶은 배우고 성장하며, 제 분야에서 차이를 만드는 데에 힘쓴 여정이었습니다. 서울 대학교 국제무역학과의 학생으로서 저는 무역 및 비즈니스에 대한 열정과 글로벌 시장에 대한 이해도를 키웠습니다. 이는 제가 삼성과 함께하는 경력의 기반이 되었으며, 저는 그곳에서 거의 20년간 무역을 담당하는 중요한 역할을 수행하며 우리 기업의 성장에 기여했습니다.

삼성에서의 시간 동안, 저는 회사의 성공과 더 큰 선을 이루기 위해 저의 기술과 재능을 활용하고자 했습니다. 무역 파트에서 일하면서 국제 비즈니스 세계와의 교량 역할을 하며 새로운 시장을 개척하고 고객과의 관계를 맺는 데 도움을 주었습니다. 제 업무 중 가장 기억에 남는 순간은 새로운 시장으로의 확장이었는데, 이는 제가 다양한 문화와 관습에 대해 배울 기회를 제공했습니다.

특히 한 번의 프로젝트가 저에게 큰 영향을 미쳤습니다. 우리는 신규 시장에 진출하여 현지 기업들과 파트너십을 맺고 그 지역에서 우리 회사의 입지를 확립하는 것을 목표로 했습니다. 이 과정에서 저는 새로운 도전과 장애물에 직면했고, 이는 저를 더욱 강인하고 적응력 있게 만들었습니다. 하지만 또한 성공의 달콤함을 맛볼 수 있었고, 그것이 제 커리어에 대한 열정을 불태웠습니다.

삼성에서의 시간 외에도, 비영리 단체와 자원봉사 프로젝트에도 참여했습니다. 이러한 활동들은 저를 더 큰 선을 위해 일하며 저의 기술과 재능을 공유할 기회를 제공했으며, 사회에 긍정적인 영향을 미치고자 하는 저의 욕구를 심어주었습니다. 저는 이것이 우리 모두의 진보에 기여하는 가장 보람찬 방법이라고 믿습니다.

결론적으로 제 커리어는 많은 성취와 영향력 있는 순간들로 가득 차 있었습니다. 무역 분야의 전문가로서의 역할, 새로운 시장을 개척한 것 그리고 비영리 단체에서의 봉사는 저에게 소중한 경험을 제공했으며 저의 성장에 기여했습니다. 이러한 경험들은 제 삶의 의미를 더 깊이 느끼게 해주었으며, 계속해서 배우고 성장하며 우리 세계에 긍정적인 영향을 미치고자 하는 욕구를 심어주었

In [244]:
query6 = 'PDF 문서을 통해 업적, 지헤의 말과 교훈을 회고해줘'
docs6 = retriever.get_relevant_documents(query6)

In [245]:
print(docs6)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\

In [246]:
template6 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [247]:
prompt6 = ChatPromptTemplate.from_template(template6)

In [248]:
chain6 = prompt6 | llm | StrOutputParser()
response6 = chain6.invoke({'context': (format_docs(docs6)), 'question':query6})
print(response6)

"살아라"가 저의 좌우명입니다. 이 말을 항상 기억하며 어려운 순간에 자신을 돌아보고 성찰할 수 있었습니다. 제가 남기고 싶은 유산은 아이들에게 사랑과 정직의 가치를 전하고 그들이 바른 길을 선택하여 이 세상에서 건강하게 행복하게 살아갈 수 있도록 제 삶의 경험을 통해 가르침을 남기는 것입니다. 미래에 대한 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것과 이 세상이 조금 더 평화롭게 되어 사람들이 서로를 이해하고 존중하는 곳이 되는 것입니다.

지금까지의 인생을 한마디로 표현한다면 "감사하다"라고 표현할 수 있습니다. 모든 순간이 감사했고, 그 덕분에 현재의 제가 존재한다고 생각합니다. 죽음을 대할 때 삶의 자연스러운 부분으로 생각하며 지금 이 순간을 최대한 살아가려고 노력하고 있습니다. 저의 인생에서 가장 기억에 남는 여행은 어려운 이들을 위해 헌신했던 사람들의 삶을 목격한 것이었고, 그들의 헌신은 저에게 큰 영감을 주었습니다.

여러분은 어떠신가요? 여러분의 인생에서 가장 기억에 남고 의미 있는 경험들은 무엇인가요? 그리고 어떻게 이러한 경험들이 여러분이 되고 싶은 사람으로 만들어주었나요? 여러분의 이야기를 나눠주시면 정말 좋겠습니다. 함께 배우고 성장해나가면서 우리 모두의 삶을 더욱 풍요롭게 만들 수 있을 것이라 생각합니다.

감사합니다. 여러분들의 생각과 의견을 기다리겠습니다!"살아라"가 저의 좌우명입니다. 이 말을 항상 기억하며 어려운 순간에 자신을 돌아보고 성찰할 수 있었습니다. 제가 남기고 싶은 유산은 아이들에게 사랑과 정직의 가치를 전하고 그들이 바른 길을 선택하여 이 세상에서 건강하게 행복하게 살아갈 수 있도록 제 삶의 경험을 통해 가르침을 남기는 것입니다. 미래에 대한 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것과 이 세상이 조금 더 평화롭게 되어 사람들이 서로를 이해하고 존중하는 곳이 되는 것입니다.

지금까지의 인생을 한마디로 표현한다면 "감사하다"라고 표현할 수 있습니다. 모든 순간이 감사했고, 그 덕분에 현재의 제가

In [249]:
query7 = 'PDF 문서을 통해 유산과 마지막 생각들을 회고해줘'
docs7 = retriever.get_relevant_documents(query7)

In [250]:
print(docs7)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\

In [251]:
template7 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [252]:
prompt7 = ChatPromptTemplate.from_template(template7)

In [254]:
chain7 = prompt7 | llm | StrOutputParser()
response7 = chain7.invoke({'context': (format_docs(docs7)), 'question':query7})
print(response7)

"살아라"는 제 좌우명이며, 어려움 속에서도 항상 자신을 돌아보고 성찰할 수 있도록 도와주었습니다. 제가 남기고 싶은 유산은 아이들에게 사랑과 정직의 가치를 전하는 것입니다. 그들이 삶에서 올바른 선택을 할 수 있도록, 제 경험을 통해 교훈을 전하고 싶습니다.

제 인생에서 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것이며, 또한 이 세상이 조금 더 평화로워지고 사람들이 서로를 이해하며 존중하는 곳이 되기를 바랍니다.

지금까지의 인생을 한 단어로 표현한다면 "감사하다"라고 할 것입니다. 모든 순간이 소중했고 그 감사함을 통해 현재의 제가 되었습니다.

죽음에 대해 저는 그것이 삶의 자연스러운 부분이라고 생각합니다. 지금 이 순간을 최선으로 살며 후회 없이 살아가려고 노력합니다. 세상을 더 나은 곳으로 남기고자 하며, 제 유산은 사랑과 정직의 가치를 실천하는 것입니다.

이것이 제가 지금까지 살아온 삶이며 앞으로도 계속 그렇게 살아갈 것입니다."살아라"는 제 좌우명이며, 어려움 속에서도 항상 자신을 돌아보고 성찰할 수 있도록 도와주었습니다. 제가 남기고 싶은 유산은 아이들에게 사랑과 정직의 가치를 전하는 것입니다. 그들이 삶에서 올바른 선택을 할 수 있도록, 제 경험을 통해 교훈을 전하고 싶습니다.

제 인생에서 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것이며, 또한 이 세상이 조금 더 평화로워지고 사람들이 서로를 이해하며 존중하는 곳이 되기를 바랍니다.

지금까지의 인생을 한 단어로 표현한다면 "감사하다"라고 할 것입니다. 모든 순간이 소중했고 그 감사함을 통해 현재의 제가 되었습니다.

죽음에 대해 저는 그것이 삶의 자연스러운 부분이라고 생각합니다. 지금 이 순간을 최선으로 살며 후회 없이 살아가려고 노력합니다. 세상을 더 나은 곳으로 남기고자 하며, 제 유산은 사랑과 정직의 가치를 실천하는 것입니다.

이것이 제가 지금까지 살아온 삶이며 앞으로도 계속 그렇게 살아갈 것입니다.
